gives you tools to stear your investigations in the right direction whenever your automatice retraining pipeline looses capacity to deliver ever-better performing model versions.

some circumstances could legitimate to include preprocessing to hyperparam tuning (for instance, to dynamically evaluate different values for "num_feature1/buckets_count" )

in some situations, model blessing can also be conditionnal on carateristics such as "inference latency".

<hr />

# setup

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os, json

In [3]:
# WandB API key
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [0]:
from IPython import get_ipython

def reset_jupyter_history():
    """
    reset cell numbering
    """

    import os
    import sqlite3
    from traitlets.config import Config
    from IPython.core.history import HistoryManager
    
    ip = get_ipython()
    
    if ip is not None:
        # Disable history temporarily
        if hasattr(ip, 'history_manager'):
            ip.history_manager.enabled = False
        
        # Get the path to the history database
        hist_file = os.path.expanduser('~/.ipython/profile_default/history.sqlite')
        if hasattr(ip, 'history_manager') and hasattr(ip.history_manager, 'hist_file'):
            hist_file = ip.history_manager.hist_file
        
        if os.path.exists(hist_file):
            # Close any existing connection
            if hasattr(ip, 'history_manager') and hasattr(ip.history_manager, 'db') and ip.history_manager.db is not None:
                ip.history_manager.db.close()
            
            # Truncate the database
            conn = sqlite3.connect(hist_file)
            conn.execute("DELETE FROM history")
            conn.execute("DELETE FROM sqlite_sequence WHERE name='history'")
            conn.commit()
            conn.close()
        
        # Create a new Config object
        c = Config()
        c.HistoryManager.hist_file = hist_file
        
        # Create a new HistoryManager instance
        new_history_manager = HistoryManager(shell=ip, config=c)
        
        # Assign the new history manager to the shell
        ip.history_manager = new_history_manager
        
        # Enable history
        ip.history_manager.enabled = True

        ip.kernel.shell.execution_count = 0
        
        print("Jupyter history has been reset.")
    else:
        print("This function must be run within a Jupyter notebook.")

reset_jupyter_history()
get_ipython().kernel.shell.execution_count

Jupyter history has been reset.


0

<hr />

# Generate data

In [4]:
from retrain_pipelines.dataset import DatasetType, pseudo_random_generate

num_samples = 10_000 # 30 # 500 # 1_500 # 
data = pseudo_random_generate(DatasetType.TABULAR_REGRESSION, num_samples)
print(data.head())
# save to file
data.to_csv(os.path.realpath(os.path.join('..', 'data', 'synthetic_classif_tab_data_continuous.csv')), index=False)

  categ_feature0  num_feature1  num_feature2  num_feature3  num_feature4  \
0         value8           0.0           1.0           4.0           0.9   
1         value7           0.0           1.0           4.5           0.7   
2         value1           0.8           1.0           3.6           0.9   
3         value7           0.0           1.0           1.1           0.9   
4         value6           0.6           1.0           7.7           0.7   

   target  
0   0.463  
1   0.178  
2   0.273  
3   0.497  
4   0.421  


<hr />

# Metaflow Run

Chosse which domain shall be considered for the HP tuning grid search&nbsp;:

In [5]:
pipeline_hp_grid = {
    "boosting_type": ["gbdt"],
    "num_leaves": [10],
    "learning_rate": [0.01],
    "n_estimators": [2],
}
os.environ['pipeline_hp_grid'] = str(json.dumps(pipeline_hp_grid)).replace("\n", "")
print(os.environ['pipeline_hp_grid'])

{"boosting_type": ["gbdt"], "num_leaves": [10], "learning_rate": [0.01], "n_estimators": [2]}


In [5]:
import itertools

pipeline_hp_grid = {
    "boosting_type": ["gbdt"],
    "num_leaves": [75, 100, 125],
    "learning_rate": [0.01],
    "n_estimators": [75, 125],
    "lambda_l1": [0, 0.05],
    "lambda_l2": [0.1, 0.2, 0.3],
    "bagging_fraction": [1, 0.95],
}
os.environ['pipeline_hp_grid'] = str(json.dumps(pipeline_hp_grid))
print(os.environ['pipeline_hp_grid'])
# combinatons count :
len([dict(zip(pipeline_hp_grid.keys(), v))
     for v in itertools.product(*pipeline_hp_grid.values())])

{"boosting_type": ["gbdt"], "num_leaves": [75, 100, 125], "learning_rate": [0.01], "n_estimators": [75, 125], "lambda_l1": [0, 0.05], "lambda_l2": [0.1, 0.2, 0.3], "bagging_fraction": [1, 0.95]}


72

## Run flow

In [2]:
%load_ext retrain_pipelines.local_launcher_magic

In [14]:
%retrain_pipelines_local retraining_pipeline.py run --help

 retrain-pipelines 0.1.0 
Metaflow 2.10.0 executing LightGbmHpCvWandbFlow for user:organization
Usage: retraining_pipeline.py run 
           [OPTIONS]

  Run the workflow locally.

Options:
  --data_file FILEPATH            Path to the
                                  input data file
                                  [default: /mnt/d
                                  /jupyter_noteboo
                                  ks/job_hunt/AWS/
                                  fresh_start/samp
                                  le_pipelines/dat
                                  a/synthetic_clas
                                  sif_tab_data_con
                                  tinuous.csv]

  --buckets_param JSON            Bucketization to
                                  be applied on
                                  raw numerical
                                  feature(s). dict
                                  of optional
                                  pairs of  featur
             

In [3]:
%retrain_pipelines_local retraining_pipeline.py run \
    --data_file "../data/synthetic_classif_tab_data_continuous.csv" \
    --buckets_param '{"num_feature1": 100, "num_feature2": 50}' \
    --pipeline_hp_grid "${pipeline_hp_grid}" \
    --cv_folds 2 \
    --max-workers 4 \
    --dask_partitions 4 \
    --wandb_run_mode disabled
#\
    #--pipeline_card_artifacts_path "." \
    #--preprocess_artifacts_path "." \

 retrain-pipelines 0.1.0 
Metaflow 2.10.0 executing LightGbmHpCvWandbFlow for user:organization
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
Including file /mnt/d/jupyter_notebooks/job_hunt/AWS/fresh_start/sample_pipelines/LightGBM_hp_cv_WandB/../data/synthetic_classif_tab_data_continuous.csv of size 282KB 
2024-10-18 13:13:43.226 Workflow starting (run-id 992):
2024-10-18 13:13:43.793 [992/start/30041 (pid 12546)] Task is starting.
2024-10-18 13:13:47.606 [992/start/30041 (pid 12546)] LightGbmHpCvWandbFlow - 992
2024-10-18 13:13:47.636 [992/start/30041 (pid 12546)] /home/organization/local_datastore/.metaflow/LightGbmHpCvWandbFlow/992
2024-10-18 13:13:52.959 [992/start/30041 (pid 12546)] Task finished successfully.
2024-10-18 13:13:53.822 [992/eda/30042 (pid 12584)] Task is starting.
2024-10-18 13:14:01.590 [992/eda/30042 (pid 12584)] Task finished successfully.
2024-10-18 13:14:02.237 [992/preprocess_data/3004

2024-10-18 13:14:45.157 [992/cross_validation/30050 (pid 12807)] Foreach yields 2 child steps.
2024-10-18 13:14:45.157 [992/cross_validation/30050 (pid 12807)] Task finished successfully.
2024-10-18 13:14:45.727 [992/cross_validation/30054 (pid 12899)] Task is starting.
2024-10-18 13:14:45.731 [992/cross_validation/30051 (pid 12818)] Foreach yields 2 child steps.
2024-10-18 13:14:45.731 [992/cross_validation/30051 (pid 12818)] Task finished successfully.
2024-10-18 13:14:46.031 [992/cross_validation/30055 (pid 12922)] Task is starting.
2024-10-18 13:14:46.040 [992/cross_validation/30052 (pid 12828)] Foreach yields 2 child steps.
2024-10-18 13:14:46.041 [992/cross_validation/30052 (pid 12828)] Task finished successfully.
2024-10-18 13:14:46.285 [992/cross_validation/30056 (pid 12938)] Task is starting.
2024-10-18 13:14:46.286 [992/cross_validation/30053 (pid 12839)] Foreach yields 2 child steps.
2024-10-18 13:14:46.287 [992/cross_validation/30053 (pid 12839)] Task finished successfully.

2024-10-18 13:15:17.666 [992/cross_validation/30065 (pid 13130)] Foreach yields 2 child steps.
2024-10-18 13:15:17.666 [992/cross_validation/30065 (pid 13130)] Task finished successfully.
2024-10-18 13:15:17.898 [992/cross_validation/30069 (pid 13220)] Task is starting.
2024-10-18 13:15:21.518 [992/cross_validation/30066 (pid 13181)] 30066 : {'boosting_type': 'gbdt', 'num_leaves': 100, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.2, 'bagging_fraction': 1}
2024-10-18 13:15:21.735 [992/cross_validation/30067 (pid 13197)] 30067 : {'boosting_type': 'gbdt', 'num_leaves': 100, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.2, 'bagging_fraction': 0.95}
2024-10-18 13:15:22.654 [992/cross_validation/30068 (pid 13209)] 30068 : {'boosting_type': 'gbdt', 'num_leaves': 100, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.3, 'bagging_fraction': 1}
2024-10-18 13:15:22.856 [992/cross_validation/30069 (pid 13220)

2024-10-18 13:16:40.085 [992/cross_validation/30081 (pid 13575)] 30081 : {'boosting_type': 'gbdt', 'num_leaves': 125, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.3, 'bagging_fraction': 0.95}
2024-10-18 13:16:48.629 [992/cross_validation/30078 (pid 13520)] Foreach yields 2 child steps.
2024-10-18 13:16:48.629 [992/cross_validation/30078 (pid 13520)] Task finished successfully.
2024-10-18 13:16:54.692 [992/training_job/30082 (pid 13646)] Task is starting.
2024-10-18 13:17:02.323 [992/cross_validation/30079 (pid 13551)] Foreach yields 2 child steps.
2024-10-18 13:17:02.323 [992/cross_validation/30079 (pid 13551)] Task finished successfully.
2024-10-18 13:17:05.030 [992/training_job/30083 (pid 13682)] Task is starting.
2024-10-18 13:17:05.032 [992/cross_validation/30080 (pid 13562)] Foreach yields 2 child steps.
2024-10-18 13:17:05.032 [992/cross_validation/30080 (pid 13562)] Task finished successfully.
2024-10-18 13:17:05.036 [992/cross_validation/30081 (p

2024-10-18 13:17:46.018 [992/training_job/30083 (pid 13682)] [LightGBM] [Info] Connected to rank 1
2024-10-18 13:17:46.018 [992/training_job/30083 (pid 13682)] [LightGBM] [Info] Connected to rank 2
2024-10-18 13:17:46.018 [992/training_job/30084 (pid 13693)] [LightGBM] [Info] Binding port 37623 succeeded
2024-10-18 13:17:46.018 [992/training_job/30083 (pid 13682)] [LightGBM] [Info] Connected to rank 3
2024-10-18 13:17:46.018 [992/training_job/30084 (pid 13693)] [LightGBM] [Info] Listening...
2024-10-18 13:17:46.018 [992/training_job/30083 (pid 13682)] [LightGBM] [Info] Connected to rank 0
2024-10-18 13:17:46.018 [992/training_job/30084 (pid 13693)] [LightGBM] [Info] Trying to bind port 51643...
2024-10-18 13:17:46.019 [992/training_job/30083 (pid 13682)] [LightGBM] [Info] Connected to rank 2
2024-10-18 13:17:46.019 [992/training_job/30084 (pid 13693)] [LightGBM] [Info] Binding port 51643 succeeded
2024-10-18 13:17:46.019 [992/training_job/30083 (pid 13682)] [LightGBM] [Info] Connected 

...

2024-10-18 13:32:00.452 [992/cross_validation_agg/30164 (pid 25280)] mean : 0.11765145286994981, std : 0.001682830590263154, median : 0.11765145286994981
2024-10-18 13:32:00.452 [992/cross_validation_agg/30164 (pid 25280)] hp values lead to an rmse of {'boosting_type': 'gbdt', 'num_leaves': 75, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.3, 'bagging_fraction': 1, 'rmse': 0.11765145286994981}
2024-10-18 13:32:01.035 [992/cross_validation_agg/30165 (pid 25291)] hp_cv_rmses : [0.11597441247861504, 0.1193672314527947],
2024-10-18 13:32:01.035 [992/cross_validation_agg/30165 (pid 25291)] mean : 0.11767082196570486, std : 0.0016964094870898297, median : 0.11767082196570486
2024-10-18 13:32:01.036 [992/cross_validation_agg/30165 (pid 25291)] hp values lead to an rmse of {'boosting_type': 'gbdt', 'num_leaves': 75, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.3, 'bagging_fraction': 0.95, 'rmse': 0.11767082196570486}
2024-10-18 13:

2024-10-18 13:32:20.507 [992/cross_validation_agg/30173 (pid 25477)] mean : 0.1177458235679571, std : 0.0016597525274969424, median : 0.1177458235679571
2024-10-18 13:32:20.507 [992/cross_validation_agg/30173 (pid 25477)] hp values lead to an rmse of {'boosting_type': 'gbdt', 'num_leaves': 100, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.1, 'bagging_fraction': 0.95, 'rmse': 0.1177458235679571}
2024-10-18 13:32:20.966 [992/cross_validation_agg/30174 (pid 25488)] hp_cv_rmses : [0.11608911966078818, 0.11941905650375216],
2024-10-18 13:32:20.966 [992/cross_validation_agg/30174 (pid 25488)] mean : 0.11775408808227017, std : 0.001664968421481991, median : 0.11775408808227017
2024-10-18 13:32:20.966 [992/cross_validation_agg/30174 (pid 25488)] hp values lead to an rmse of {'boosting_type': 'gbdt', 'num_leaves': 100, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0.05, 'lambda_l2': 0.2, 'bagging_fraction': 1, 'rmse': 0.11775408808227017}
2024-10-18 13:

2024-10-18 13:32:43.173 [992/cross_validation_agg/30183 (pid 25702)] mean : 0.11778086453006702, std : 0.0017326815871085838, median : 0.11778086453006702
2024-10-18 13:32:43.173 [992/cross_validation_agg/30183 (pid 25702)] hp values lead to an rmse of {'boosting_type': 'gbdt', 'num_leaves': 125, 'learning_rate': 0.01, 'n_estimators': 50, 'lambda_l1': 0, 'lambda_l2': 0.3, 'bagging_fraction': 0.95, 'rmse': 0.11778086453006702}
2024-10-18 13:32:46.740 [992/cross_validation_agg/30180 (pid 25651)] Task finished successfully.
2024-10-18 13:32:47.485 [992/cross_validation_agg/30184 (pid 25743)] Task is starting.
2024-10-18 13:32:48.482 [992/cross_validation_agg/30181 (pid 25662)] Task finished successfully.
2024-10-18 13:32:48.678 [992/cross_validation_agg/30185 (pid 25772)] Task is starting.
2024-10-18 13:32:48.689 [992/cross_validation_agg/30182 (pid 25685)] Task finished successfully.
2024-10-18 13:32:48.894 [992/cross_validation_agg/30186 (pid 25782)] Task is starting.
2024-10-18 13:32:4

2024-10-18 13:33:09.497 [992/best_hp/30190 (pid 25888)] 30070          gbdt         125           0.01            50       0.00        0.1              1.00 0.117761
2024-10-18 13:33:09.498 [992/best_hp/30190 (pid 25888)] 30071          gbdt         125           0.01            50       0.00        0.1              0.95 0.117756
2024-10-18 13:33:09.498 [992/best_hp/30190 (pid 25888)] 30066          gbdt         100           0.01            50       0.05        0.2              1.00 0.117754
2024-10-18 13:33:09.498 [992/best_hp/30190 (pid 25888)] 30068          gbdt         100           0.01            50       0.05        0.3              1.00 0.117753
2024-10-18 13:33:09.498 [992/best_hp/30190 (pid 25888)] 30069          gbdt         100           0.01            50       0.05        0.3              0.95 0.117748
2024-10-18 13:33:09.498 [992/best_hp/30190 (pid 25888)] 30065          gbdt         100           0.01            50       0.05        0.1              0.95 0.117746
2024

2024-10-18 13:33:21.829 [992/train_model/30191 (pid 25924)] [LightGBM] [Info] Connected to rank 3
2024-10-18 13:33:21.829 [992/train_model/30191 (pid 25924)] [LightGBM] [Info] Local rank: 3, total number of machines: 4
2024-10-18 13:33:21.829 [992/train_model/30191 (pid 25924)] [LightGBM] [Info] Local rank: 1, total number of machines: 4
2024-10-18 13:33:21.829 [992/train_model/30191 (pid 25924)] [LightGBM] [Info] Local rank: 0, total number of machines: 4
2024-10-18 13:33:21.830 [992/train_model/30191 (pid 25924)] [LightGBM] [Info] Local rank: 2, total number of machines: 4
2024-10-18 13:33:21.830 [992/train_model/30191 (pid 25924)] [LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
2024-10-18 13:33:21.830 [992/train_model/30191 (pid 25924)] [LightGBM] [Warning] lambda_l1 is set=0.05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.05
2024-10-18 13:33:21.831 [992/train_model/30191 (pid 25924)] [LightGBM] [Wa

In [ ]:
%retrain_pipelines_local retraining_pipeline.py resume pipeline_card

# Inspectors

We can programatically interact with the Metaflow service using the `metaflow`python package. To connect the package with our self-hosted metaflow service, we simply need to set a couple environment variables before importing it&nbsp;:

In [ ]:
mf_flow_name = 'LightGbmHpCvWandbFlow'

## local Metaflow SDK

In [ ]:
from retrain_pipelines.frameworks import local_metaflow as metaflow

In [ ]:
metaflow.Task("LightGbmHpCvWandbFlow/897/pipeline_card/27936",  attempt=0)['encoder'].data.__dict__

Inspectors are convenience methods that abstract away some of the logic to get access to Metaflow pipeline-run artifacts.

## local custom card explorer

In [ ]:
from retrain_pipelines.inspectors import browse_local_pipeline_card

In [ ]:
#browse_local_pipeline_card(mf_flow_name, mf_run_id=633, verbose=True)
browse_local_pipeline_card(mf_flow_name)
#browse_local_pipeline_card(mf_run_id=797, verbose=True)

<hr />

## WandB

In [ ]:
from retrain_pipelines.inspectors import get_execution_source_code

In [ ]:
help(get_execution_source_code)

In [ ]:
artifact_files = get_execution_source_code(mf_run_id=978)

for file in artifact_files:
    print(f" - {file.name} {file.url}")
    # file.download(replace=True)

In [ ]:
from retrain_pipelines.inspectors import explore_source_code
# download and open file explorer
explore_source_code(mf_run_id=978)

<hr />

## hp_cv_inspector

The herein retraining pipeline rely on Dask for data-parallel training. Each Cross-Validation fold of each set of hyperparameter value is trained using a subset of the dataset, parallelized accross workers.

We look into pipeline runs from the perspective of detailed training logs during hyperparameter tuning.

First, focusing on the best-performing set of hyperparameters values&nbsp;:

In [ ]:
from inspectors import plot_run_cv_history
plot_run_cv_history(best_cv=True) #mf_run_id=432, 

Now, looking at all sets of hyperparameter values evaluated&nbsp;:

In [ ]:
from inspectors import plot_run_all_cv_tasks
plot_run_all_cv_tasks()#mf_run_id=432)

# dependencies

In [ ]:
%pip list

In [ ]:
%pip list --format=freeze > requirements.txt

# END